# Kats 204 Forecasting with Meta Learning


Suppose we have a time series and we are looking to build the best possible forecast (with respect to a predefined error metric such as mean absolute error) from the following list of candidate models (and possibly other forecasting models in Kats too):
* ARIMA
* SARIMA
* Holt-Winters
* Prophet
* Theta
* STLF



For a single time series, it is straightforward to to do hyperparameter tuning for each of the candidate models with this time series, calculate the error metric, and choose the model that minimizes the error metric.  We have discussed this methodology in detail in Kats 201.  Our basic metadata object, `GetMetaData`, which we will introduce below, also does this calculation to find the best forecasting model for a single time series.

However, when we are working with a large number of time series, repeating this process quickly becomes intractable, and for that, we include a meta-learning framework for forecasting.  There are two key model classes, plus one optional one, in our meta-learning framework:

1. `MetaLearnModelSelect`: Given the metadata for a time series, predict the best model family (from the candidate models of interest) to forecast the series.  This model is a random forest by default.
2. `MetaLearnHPT`: Given a time series and a model type, predict the best parameters for this model.  This model is a neutral network.  
3. `MetaLearnPredictability` (optional): Given the metadata for a time series, predict if it is "predictable", i.e. if it is possible to forecast with a threshold error.  This model is a random forest by default.

For each of these models, you can use labeled training data to build a model or you load a pre-trained model from a file path.  

We use the `GetMetaData` object to represent the metadata for a time series in `MetaLearnModelSelect` and `MetaLearnPredictability`.  This tutorial begins with an introduction to the `GetMetaData` object.  Since this object is heavily dependent on `TsFeatures`, if you are not familiar `TsFeatures`, you should check out Kats 203 prior to continuing with this tutorial.  

Next we will use labeled time series data from the `m3_meta_data.csv` file to show how to use the `MetaLearnPredictability`, `MetaLearnModelSelect` and `MetaLearnPredictability`.  


The sample data in `m3_meta_data.csv` is very small, with 78 labeled examples, so the examples we provide here will not be highly accurate, but they will show you the proper workflow for using the meta-learning framework for forecasting in Kats.


The full table of contents for Kats 204 is as follows:
1. Introduction to `GetMetaData`
2. Determining Predictability with `MetaLearnPredictability`
3. Model Selection with `MetaLearnModelSelect`
4. Hyperparameter Tuning with `MetaLearnHPT`


## 1. Introduction to `GetMetaData`

The `GetMetaData` class generates the metadata for any time series.  There are three key components to the the metadata for a time series:
1. `features`: the `TsFeatures` dictionary for the time series
2. `hpt_res`: a dictionary giving the best hyperparameters for each candidate model and the corresponding error metric for the time series 
3. `best_model`: the name of the model with the smallest error metric

The default error metric is mean absolute error (mae) but this can be controlled with the `error_method` argument in `GetMetaData`.  

The list of candidate models that we consider is controlled by the `all_models` argument in `GetMetaData`, which is a dictionary with string names of the candidate models as keys and corresponding model classes as value.

with keys equal to the string names of the models as keys  and values equal to the corresponding model class.  The keys in `hpt_res` and the value of `best_model` come from the keys of the `all_models` dictionary.  The default value of `all_models` will include the following six models.

1. ARIMA
2. SARIMA
3. Holt-Winters
4. Prophet`
5. Theta
6. STLF

Our first example uses the `air_passengers` data set.  We show how to get the metadata for this time series.  We start by loading the time series into a `TimeSeriesData` object.

In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore')
sys.path.append("../")

from kats.consts import TimeSeriesData
air_passengers_df = pd.read_csv("../kats/data/air_passengers.csv")

air_passengers_df.columns = ["time", "value"]
air_passengers_ts = TimeSeriesData(air_passengers_df)

Now we can construct the `GetMetaData` object with all of the default settings for the `air_passengers` data set time series like follows.

In [2]:
from kats.models.metalearner.get_metadata import GetMetaData

# create an object MD of class GetMetaData
MD = GetMetaData(data=air_passengers_ts)

Let's take a look at the `all_models` dictionary that is used by default here.  You are allowed to specify your own `all_models` dictionary as long as all the values are classes that extends the abstract class `kats.models.Model`.

In [3]:
MD.all_models

{'arima': kats.models.arima.ARIMAModel,
 'holtwinters': kats.models.holtwinters.HoltWintersModel,
 'prophet': kats.models.prophet.ProphetModel,
 'theta': kats.models.theta.ThetaModel,
 'stlf': kats.models.stlf.STLFModel,
 'sarima': kats.models.sarima.SARIMAModel}

The `all_params` dictionary will have the same keys as the `all_models` dictionary, and the values are the corresponding parameter class (i.e. a class that extends the class `kats.const.Params`)

In [4]:
MD.all_params

{'arima': kats.models.arima.ARIMAParams,
 'holtwinters': kats.models.holtwinters.HoltWintersParams,
 'prophet': kats.models.prophet.ProphetParams,
 'theta': kats.models.theta.ThetaParams,
 'stlf': kats.models.stlf.STLFParams,
 'sarima': kats.models.sarima.SARIMAParams}

Now we can use the `get_meta_data` function to calculate all the metadata and output the result as a dictionary.

In [5]:
# get meta data as a dictionary
air_passengers_metadata = MD.get_meta_data()

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check m

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:548: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:548: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

/usr

Let's take a look at the keys of the metadata dictionary.

In [6]:
air_passengers_metadata.keys()

dict_keys(['hpt_res', 'features', 'best_model', 'search_method', 'error_method'])

We explained what `features`, `hpt_res` and `best_model` are above.  This dictionary also includes the `search_method` and `error_method`, which will just be the default values in this case.  We can see these as follows.

In [7]:
print(f"search_method: {air_passengers_metadata['search_method']}")
print(f"error_method: {air_passengers_metadata['error_method']}")

search_method: RandomSearch
error_method: mae


The keys of the `hpt_res` dictionary are name of the candidate model families; they should be the same as the keys for the `all_models` and `all_parameters` dictionaries. 

In [8]:
air_passengers_metadata['hpt_res'].keys()

dict_keys(['arima', 'holtwinters', 'prophet', 'theta', 'stlf', 'sarima'])

The values of the `hpt_res` dictionary are two-element tuples.  The first element is gives the hyperparameters that minimize the error metric.  The second element gives the corresponding minimum error metric.  Let's take a look at these values for ARIMA:

In [9]:
air_passengers_metadata['hpt_res']['arima']

({'p': 5, 'd': 1, 'q': 1}, 0.07938630340450795)

We can sort the different methods by their error metric as follows:

In [10]:
methods = list(air_passengers_metadata['hpt_res'].keys())
sorted(methods, key = lambda m: air_passengers_metadata['hpt_res'][m][1])

['prophet', 'stlf', 'holtwinters', 'theta', 'arima', 'sarima']

This suggests that Prophet has the lowest error metric.  Let's confirm that this is what `best_model` indicates:

In [11]:
air_passengers_metadata['best_model']

'prophet'

We constructed the `GetMetaData` object for the `air_passengers` data set with all of the default settings.  To wrap up this introduction to `GetMetaData`, let's take a look at the full set of attributes that can be used to initialize `GetMetadata`.

This is the only required attributed: 
* **data**: TimeSeriesData, the time series for which we calculate the metadata

The following attributes are all optional:
* **all_models**: `Dict[str, m.Model]`, a dictionary for the candidate model classes.  The key is a string naming the model and each value is a corresponding model class (i.e. a class that extends the abstract class `kats.models.Model`).
* **all_params**: `Dict[str, Params]`, a dictionary for the candidate model parameter classes.  The keys are the same as the keys for `all_models` and each value is a corresponding parameter class (i.e. a class that extends the class `kats.const.Params`).
* **min_length**: int, the minimal length of time series. We raise a value error if the length of `data` is smaller than `min_length`.  The default value of `min_length` is 30.
* **scale**: bool, Whether to rescale the time series by its maximum value; default is true.
* **method**: SearchMethodEnum, Search method for hyper-parameters tuning; default is random search in the default parameter space
* **executor**: Callable, A parallel executor for parallel processing.  By default, we use Python's native multiprocessing implementation.
* **error_method**: str, Type of error metric.  Options are `'mape`', `'smape`',`'mae`', `'mase`', `'mse`', `'rmse`';  default is `'mae'`.
* **num_trials**: int, Number of trials for hyperparameter search; default is 5
* **num_arm**: optional Number of arms in hyperparameter search; default is 4.

## 2. Determining Predictability with `MetaLearnPredictability`

In this tutorial, we pre-collected meta-data from m3 monthly data to demonstrate how to build and use meta-learning models. Notice that we are using a very small meta-data dataset (i.e., 78 records) and hence the performance of meta-learning models may not be ideal. In real applications, please considering using a relatively large meta-data dataset to ensure good performance.

Before using meta-learning models for model and hyper-parameters forecasting, we would like to know whether a target time series can be easily fitted by a simple model or should be taken care of by extra human efforts. If a time series is taken as unpredictable, then the results given by our meta-learning framework may not be satisfying. Whether a time series is predictable or not is defined by user via specifying the "threshold" parameter for a target error metric (i.e., the model will take the time series with error metric>=threshold as unpredicable). This step is optional.

Parameters for MetaLearnPredictability() class:
* **metadata**: Optional; A list of dictionaries representing the meta-data of time series (e.g., the meta-data generated by GetMetaData object).Each dictionary d must contain at least 3 components: 'hpt_res', 'features' and 'best_model'. d['hpt_res'] represents the best hyper-parameters for each candidate model and the corresponding errors;
d['features'] are time series features, and d['best_model'] is a string representing the best candidate model of the corresponding time series data. metadata should not be None unless load_model is True. Default is None

* **threshold**: Optional; A float representing the threshold for the forecasting error. A time series whose forecasting error of the best forecasting model is higher than the threshold is considered as unpredictable. Default is 0.2.

* **load_model**: Optional; A boolean to specify whether or not to load a trained model. Default is None.

For the remaining examples, we use the sample data in `m3_meta_data.csv` to show how to build meta-learning models.  This sample data set contains the metadata for 78 time series.  While 78 metadata objects is certainly too few to develop an accurate meta-learning model and you should use more examples for your own meta-learning models to get high accuracy, these examples will help familiarize you with our meta-learning framework.

We load the metadata from `m3_meta_data.csv` as follows and preview it below.

In [44]:
# load the metadata into a DataFrame
meta_data_df = pd.read_csv("../kats/data/m3_meta_data.csv")

# We need to do a little pre-processing to make sure the dictionaries are represented as dictionaries
# rather than as strings.  This function will do that pre-processing.
def change_format(tmp):
    tmp['hpt_res']=eval(tmp['hpt_res'])
    tmp['hpt_res']['sarima'][0]['seasonal_order'] = eval(tmp['hpt_res']['sarima'][0]['seasonal_order'])
    tmp['features']=eval(tmp['features'])
    return tmp

meta_data_df = meta_data_df.apply(change_format, axis=1)

meta_data_df.head()

,hpt_res,features,best_model,search_method,error_method,idx
0,"{'arima': ({'p': 5, 'd': 2, 'q': 5}, 0.8932924...","{'length': 68, 'mean': 0.35392156862745106, 'v...",stlf,RandomSearch,mape,N1402
1,"{'arima': ({'p': 5, 'd': 2, 'q': 1}, 0.3989958...","{'length': 68, 'mean': 0.19995256166982928, 'v...",arima,RandomSearch,mape,N1403
2,"{'arima': ({'p': 1, 'd': 2, 'q': 3}, 0.2519500...","{'length': 68, 'mean': 0.48962530217566486, 'v...",holtwinters,RandomSearch,mape,N1404
3,"{'arima': ({'p': 2, 'd': 1, 'q': 3}, 0.3146896...","{'length': 68, 'mean': 0.2980870396939263, 'va...",stlf,RandomSearch,mape,N1405
4,"{'arima': ({'p': 3, 'd': 1, 'q': 1}, 0.1448053...","{'length': 68, 'mean': 0.5336050082683677, 'va...",arima,RandomSearch,mape,N1406


Let's convert this metadata DataFrame to a list of dictionaries.

In [ ]:
meta_data = meta_data_df.to_dict(orient='records')

In [ ]:
# load class 
from kats.models.metalearner.metalearner_predictability import MetaLearnPredictability

#take the time series with MAPE>=0.2 as unpreditable time series and initial the object
mlp=MetaLearnPredictability(meta_data, threshold=0.2)

#train a meta-learning predictability model and display the evaluation metrics on the test_set
test_metrics=mlp.train()
print("evaluation on test_set: ", test_metrics)

#test whether TSdata is predictable or not
val=mlp.pred(TSdata)
val=(True if val==1 else False)
print("TSdata is predictable: {}".format(val))


## 3. **Meta-learn Model Selection**

The MetaLearnModelSelect object helps to predict which predictive model is the most suitable one for the target time series.

Parameters for MetaLearnModelSelect():
* **metadata**: Optional; A list of dictionaries representing the meta-data of time series (e.g., the meta-data generated by GetMetaData object). Each dictionary d must contain at least 3 components: 'hpt_res', 'features' and 'best_model'. d['hpt_res'] represents the best hyper-parameters for each candidate model and the corresponding errors; d['features'] are time series features, and d['best_model'] is a string representing the best candidate model of the corresponding time series data. metadata should not be None unless load_model is True. Default is None
* **load_model**: Optional; A boolean to specify whether or not to load a trained model. Default is False.



In [ ]:
#load MetaLearnModelSelect
from kats.models.metalearner.metalearner_modelselect import MetaLearnModelSelect

#create an object mlms of class MetaLearnModelSelect
mlms = MetaLearnModelSelect(meta_data)

#get class info
counter=mlms.count_category()
print("number of time series for each model: ", counter)

#preprocessing meta-data: 
#not down-sample the meta-data to create the balanced data and standarized time series features to zero mean and unit standard deviation
mlms.preprocess(downsample=False, scale=True)

#generate features comparison plot
mlms.plot_feature_comparison(10, 35)

# generate heat map of correlation matrix of time series feature matrix
mlms.plot_corr_heatmap()

The meta-learning modelselect model is basically a multi-class classifier, and we currently support random foreset (default), GBDT, SVM, KNN, and naive Bayes. We also display the evaluation of the classifier in term of error metrics (i.e., MAPE for our example) on both the training set and test set, compared with the averaged error metric of using one model for all time series.

In [ ]:
#train a modelselect model using random forest algorithm   
results=mlms.train()

#display evaluation metrics of MetaLearnModelSelect
summary=pd.DataFrame([results['fit_error'], results['pred_error']])
summary['type']=['fit_error', 'pred_error']
summary['error_metric']='MAPE'
summary


In [ ]:
#predict forecasting model for a new time series data
pred_model = mlms.pred(TSdata)
print('Predicted forecasting model of this given time series data is: ', pred_model)


In [ ]:
#save trained model and load a pre-trained model
mlms.save_model("mlms.pkl")

#initiate a new object and load the trained model
mlms2 = MetaLearnModelSelect(load_model=True)
mlms2.load_model("mlms.pkl")

#predict forecasting model with new MetaLearnModelSelect object
pred_model = mlms2.pred(TSdata)
print('Predicted forecasting model of this given time series data is: ', pred_model)

## 4. **Meta-learning Hyper-parameter Selection**

The MetaLearnHPT model generates the suitable hyper-parameters for a target time series given a designated model. We train a multi-task neural network using the optimal hyper-parameters from the meta-data and use it to predict the hyper-parameters.

Some importand parameters for MetaLearnHPT():
* **data_x**: Optional; A pd.DataFrame of time series features. data_x should not be None unless load_model is True. Default is None.
* **data_y**: Optional; A pd.DataFrame of the corresponding best hyper-parameters. data_y should not be None unless load_model is True. Default is None.
* **categorical_idx**: Optional; A list of strings of the names of the categorical hyper-parameters. Default is None.
    If there is no categorical variable, then set categorical_idx as empty list.
* **numerical_idx**: Optional; A list of strings of the names of the numerical hyper-parameters. Default is None.
    If there is no numerical variables, then set numerical_idx as an empty list.
* **default_model**: Optional; A string of the name of the forecast model whose default settings will be used. Can be 'arima', 'sarima', 'theta', 'prophet', 'holtwinters', 'stlf' or None. Default is None.
    If None, then a customized model will be initiated.
* **load_model**: Optional; A boolean to specify whether or not to load a trained model. Default is False.

In [ ]:
# transform meta-data into pd.DataFrame
mlhpt_table={}
for m in ['arima', 'sarima', 'stlf', 'theta', 'prophet', 'holtwinters']:
    mlhpt_table[m]={'x':[], 'y':[]}
    for elm in range(len(meta_data)):
        mlhpt_table[m]['x'].append(meta_data[elm]['features'])
        mlhpt_table[m]['y'].append(meta_data[elm]['hpt_res'][m][0])

for tab in mlhpt_table:
    mlhpt_table[tab]['x']=pd.DataFrame(mlhpt_table[tab]['x'])
    mlhpt_table[tab]['y']=pd.DataFrame(mlhpt_table[tab]['y'])
    
#load MetaLearnHPT
from kats.models.metalearner.metalearner_hpt import MetaLearnHPT



### 4.1 MetaLearnHPT with default NNs

For the users who want to avoid specifying the types of hyper-parameters and the neural network structures, we provide user-friendly default neural network structures.

In [ ]:
#create an object using our default neural network (take Holt-Winter's model as an example)
tab='holtwinters'
mlhpt_holtwinters = MetaLearnHPT(
    data_x=mlhpt_table[tab]['x'],
    data_y=mlhpt_table[tab]['y'],
    default_model=tab
)

#build a multi-task neural network using our default NN structure
mlhpt_holtwinters.build_network()

#train the multi-task NN
mlhpt_holtwinters.train(lr=0.001, batch_size=20)

#plot the training curves
mlhpt_holtwinters.plot()

In [ ]:
#predict hyper-parameters using TimeSeriesData
pred=mlhpt_holtwinters.pred(TSdata)
print("predict hyper-parameters: ", pred['parameters'].iloc[0])


In [ ]:
#save the trained model
mlhpt_holtwinters.save_model("mlhpt_hw.pkl")

#initiate a new object to load the trained model
mlhpt2=MetaLearnHPT(load_model=True)
mlhpt2.load_model("mlhpt_hw.pkl")

#get prediction using the new object
pred=mlhpt2.pred(TSdata)
print("predict hyper-parameters: ", pred['parameters'].iloc[0])

### 4.2 MetaLearnHPT with customized NNs

We also give users the flexibility to specify their own NNs. 

In [ ]:
#create an object with customized structures (take Holt-Winter's model as an example).
mlhpt_holtwinters=MetaLearnHPT(
    data_x=pd.DataFrame(mlhpt_table[tab]['x']),
    data_y=pd.DataFrame(mlhpt_table[tab]['y']),	
    #specify the names of cateogrical label
    categorical_idx = [
                        "trend",
                        "damped",
                        "seasonal",
                    ],
    #specify the names of continuous label
    numerical_idx = ["seasonal_periods"]
    
)

#build the customized NN
mlhpt_holtwinters.build_network(
    #One shared one-layer NN with 50 neurons.
    n_hidden_shared=[50],
    #Each classification task has its own task-specific NN. In this example, "trend" and "dampled" both have a two-layer NN respectively
    #and "seasonal" has a one-layer NN.
    n_hidden_cat_combo=[[20, 10], [20, 10], [20]], 
    #One task-specific one-layer NN with 30 neurons for regression task.
    n_hidden_num=[30]
)

#train the customized NN
mlhpt_holtwinters.train(    
    #loss_scale is used to balance 2 types of losses: cross-entropy for classification tasks and MSE for regression tasks
    loss_scale=30,
    #learning rate
    lr=0.005,
    n_epochs=2000,
    batch_size=16,
    #supports ADAM and SGD
    method='SGD',
    #momentum in SGD.
    momentum=0,
    #early stop option.
    n_epochs_stop=50,)

#plot the training curves
mlhpt_holtwinters.plot()

In [ ]:
# make a prediction
mlhpt_holtwinters.pred(TSdata)
print("predict hyper-parameters: ", pred['parameters'].iloc[0])